In [2]:
from keras import models
from keras import layers

model = models.Sequential()
# 卷积神经网络接收形状为：(image_height, image_width, image_channels)
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

宽度和高度两个维度的尺寸通常会随着网络加深而变小。通道数量由传入Conv2D层的第一个参数所控制（32或64）。

In [3]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


In [4]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [7]:
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255.

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255.

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=20, batch_size=64)

Epoch 1/20
60000/60000 [==============================] - 32s 538us/step - loss: 0.1724 - acc: 0.9461
Epoch 2/20
60000/60000 [==============================] - 32s 531us/step - loss: 0.0501 - acc: 0.9846
Epoch 3/20
60000/60000 [==============================] - 32s 528us/step - loss: 0.0339 - acc: 0.9896
Epoch 4/20
60000/60000 [==============================] - 32s 528us/step - loss: 0.0261 - acc: 0.9918
Epoch 5/20
60000/60000 [==============================] - 32s 527us/step - loss: 0.0208 - acc: 0.9937
Epoch 6/20
60000/60000 [==============================] - 32s 527us/step - loss: 0.0164 - acc: 0.9953
Epoch 7/20
60000/60000 [==============================] - 32s 529us/step - loss: 0.0132 - acc: 0.9959
Epoch 8/20
60000/60000 [==============================] - 32s 529us/step - loss: 0.0105 - acc: 0.9970
Epoch 9/20
60000/60000 [==============================] - 32s 529us/step - loss: 0.0094 - acc: 0.9975
Epoch 10/20
60000/60000 [==============================] - 32s 530us/step - loss: 

In [9]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('test_loss =', test_loss)
print('test_acc =', test_acc)

10000/10000 [==============================] - 2s 168us/step
test_loss = 0.047333629234896486
test_acc = 0.9925


- 密集连接层和卷积层的根本区别：Dense层从输入特征空间中学到的是全局模式；而卷积层学到的是局部模式，对于图像来说，学到的就是在输入图像的二维小窗口中发现的模式。

卷积神经网络的性质：
1. 学到的模式具有平移不变性（translation invariant）。卷积神经网络在图像右下角学到某个模式之后，可在任何地方识别这个模式。而对于密集连接网络来说，如果模式出现在新的位置，它只能重新学习这个模式。
2. 可以学到模式的空间层次结构。（spatial hierarchies of patterns）。第一个卷积层将学习较小的局部模式（比如边缘），第二个卷积层将学习由第一层特征组成的更大的模式，以此类推。这使得卷积神经网络可以有效地学习越来越复杂、越来越抽象的视觉概念。

深度轴的每个维度都是一个特征（或过滤器），而2D张量output[:,:,n]是这个过滤器在输入上的响应的二维空间图。

- 使用最大池化层下采样的原因：1）减少需要处理的特征图的元素个数；2）通过让连续卷积层的观察窗口越来越大（即窗口覆盖原始输入的比例越来越大），从而引入空间过滤器的层级结构。

最合理的子采样策略：首先生成密集的特征图（通过无步进的卷积），然后观察特征每个小图块上的最大激活，而不是查看输入的稀疏窗口（通过步进卷积）或对输入图块取平均，因为后两种方法可能导致错过或淡化特征是否存在的信息。

#### 猫狗图像分类

In [18]:
import os, shutil

original_dataset_dir = 'E:\\文件管理集合\\笔记\\机器学习\\Python深度学习\\train1'
base_dir = 'E:\\文件管理集合\\笔记\\机器学习\\Python深度学习'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')
validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
test_cats_dir = os.path.join(test_dir, 'cats')
test_dogs_dir = os.path.join(test_dir, 'dogs')

# 将20000张用于训练，5000张用于验证
fnames = ['cat.{}.jpg'.format(i) for i in range(10000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_cats_dir, fname)
    shutil.copyfile(src, dst)
    
fnames = ['cat.{}.jpg'.format(i) for i in range(10000, 12500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_cats_dir, fname)
    shutil.copyfile(src, dst)

In [19]:
fnames = ['dog.{}.jpg'.format(i) for i in range(10000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_dog_dir, fname)
    shutil.copyfile(src, dst)
    
fnames = ['dog.{}.jpg'.format(i) for i in range(10000, 12500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_dogs_dir, fname)
    shutil.copyfile(src, dst)

In [21]:
print('total training cat images:', len(os.listdir(train_cats_dir)))
print('total training dog images:', len(os.listdir(train_dogs_dir)))
print('total validation cat images:', len(os.listdir(validation_cats_dir)))
print('total validation dog images:', len(os.listdir(validation_dogs_dir)))

total training cat images: 10000
total training dog images: 10000
total validation cat images: 2500
total validation dog images: 2500


In [25]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D(2, 2))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(2, 2))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(2, 2))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(2, 2))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 15, 15, 128)       147584    
__________

In [27]:
from keras import optimizers

model.compile(optimizer=optimizers.RMSprop(lr=1e-4), loss='binary_crossentropy', metrics=['acc'])

##### 数据预处理
将数据输入神经网络之前，应该将数据格式化为经过预处理的浮点数张量。
1. 读取图像文件
2. 将JPEG文件解码为RGB像素网格
3. 将这些像素网格转换为浮点数张量
4. 将像素值（0~255范围内）缩放到[0,1]区间。

In [28]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(150,150), batch_size=100, class_mode='binary')
validation_generator = test_datagen.flow_from_directory(validation_dir, target_size=(150,150), batch_size=100, class_mode='binary')

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [30]:
history = model.fit_generator(
    train_generator, 
    steps_per_epoch=200,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=50)
model.save('cats_and_dogs_1.h5')

Epoch 1/30
200/200 [==============================] - 134s 669ms/step - loss: 0.1116 - acc: 0.9602 - val_loss: 0.3698 - val_acc: 0.8770
Epoch 2/30
200/200 [==============================] - 131s 655ms/step - loss: 0.1273 - acc: 0.9512 - val_loss: 0.4098 - val_acc: 0.8470
Epoch 3/30
200/200 [==============================] - 130s 649ms/step - loss: 0.1172 - acc: 0.9512 - val_loss: 0.4339 - val_acc: 0.8470
Epoch 4/30
200/200 [==============================] - 130s 651ms/step - loss: 0.1142 - acc: 0.9567 - val_loss: 0.4945 - val_acc: 0.8450
Epoch 5/30
200/200 [==============================] - 130s 652ms/step - loss: 0.0892 - acc: 0.9670 - val_loss: 0.3402 - val_acc: 0.8870
Epoch 6/30
200/200 [==============================] - 131s 654ms/step - loss: 0.0908 - acc: 0.9665 - val_loss: 0.3693 - val_acc: 0.8740
Epoch 7/30
200/200 [==============================] - 133s 665ms/step - loss: 0.1077 - acc: 0.9565 - val_loss: 0.3991 - val_acc: 0.8560
Epoch 8/30
164/200 [=======================>....

KeyboardInterrupt: 